In [1314]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import opinion_lexicon
from nltk.corpus import sentiwordnet as swn
from pywsd import simple_lesk
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.corpus import stopwords

english_sw = set(stopwords.words("english"))

- **Ejercicio 1:** Se quiere desarrollar un evaluador de sentimientos basado en el uso de un recursos lingüístico externo, en concreto, las listas de palabras de opinión que tiene NLTK.

In [1290]:
opinion1 = "Visceral, stunning and relentless film making. Dicaprio's Herculean, almost purely physical performance" \
            "and Hardy's wide eyed intensity coupled with the almost overwhelming beauty of the landscape - those " \
            "trees, the natural light, the sun peeking through the clouds, rendered the proceedings down to savage" \
            "poetry. A hypnotic, beautiful, exhausting film."

opinion2 = "I saw this film on Friday. For the first 40 minutes involving spoken dialogue they need not have " \
            "bothered. For me the dialogue was totally unintelligible with grunting, southern states drawl, " \
            "and coarse accent that made it impossible to understand what they were saying."

opinion3 = "It was a idiotic film that produces a magnificent fascination."

In [1291]:
def tokenize_text(text):
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)
    word_tokenized = [word_tokenize(s) for s in sent_tokenized]
    return word_tokenized

In [1292]:
def classify(text):
    word_tokenized = tokenize_text(text)
    score = 0
    for s in word_tokenized:
        for w in s:
            if w in opinion_lexicon.positive() and w in opinion_lexicon.negative():
                score += 0
            elif w in opinion_lexicon.positive():
                score += 1
            elif w in opinion_lexicon.negative():
                score -= 1
    if score == 0:
        return "neutro", score
    elif score > 0:
        return "pos", score
    else:
        return "neg", score

In [1293]:
# Opinión 1
print(classify(opinion1))

('pos', 1)


In [1294]:
# Opinión 2
print(classify(opinion2))

('neg', -4)


In [1295]:
# Opinión 3
print(classify(opinion3))

('pos', 1)


---
- **Ejercicio 2:** Se pide lo mismo que en el ejercicio anterior, pero en este caso utilizando SentiWordNet (disponible en NLTK). Esta base de datos proporciona valores positivos y negativos para ciertas palabras en un rango entre -1 y 1. Se puede seguir la misma idea de algoritmo que en el caso anterior, pero hay que tener en cuenta que SentiWordNet nos proporciona puntuaciones para los diferentes sentidos que tiene una palabra. Se puede entonces considerar la puntuación de todos los sentidos de la misma palabra, restando a lo positivo la puntuación negativa. Puede ser interesante que la puntuación global se promedie de acuerdo con el número de sentidos.  Utilizar como entrada las mismas opiniones del ejercicio anterior. ¿El resultado es mejor o peor que el conseguido con el algoritmo del ejercicio 1?

In [1296]:
def classify_swn(text):
    word_tokenized = tokenize_text(text)
    total_score = 0
    for s in word_tokenized:
        for w in s:
            senti_synsets = list(swn.senti_synsets(w))
            word_score = 0
            for senti_synset in senti_synsets:
                word_score += senti_synset.obj_score()
            if len(senti_synsets) > 0:
                total_score += word_score / len(senti_synsets)
    return total_score

In [1297]:
# Opinión 1
print(classify_swn(opinion1))

27.784353633954698


In [1298]:
# Opinión 2
print(classify_swn(opinion2))

27.681840555278054


In [1299]:
# Opinión 3
print(classify_swn(opinion3))

7.056547619047619


- **Ejercicio 2.1:** Hacer una variante del ejercicio donde se tengan en cuenta primero la categoría gramatical del token para considerar únicamente los scores de los sentidos que coincidan con la categoría gramatical dada. ¿Ha mejorado el resultado o ha empeorado con respecto a versiones anteriores?

In [1300]:
# Función auxiliar para convertir un
# tag generado por nltk.pos_tag()
# en una categ. gramatical de 
# WordNet
def tag2wn(tag):
    if tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("V"):
        return wn.VERB
    elif tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("R"):
        return wn.ADV
    else:
        return None

def classify_swn_gram(text):
    word_tokenized = tokenize_text(text)
    tags = [nltk.pos_tag(s) for s in word_tokenized]
    total_score = 0
    for s in tags:
        for w, t in s:
            synsets = list(swn.senti_synsets(w, tag2wn(t)))
            word_score = 0
            for synset in synsets:
                word_score += synset.obj_score()
            if len(synsets) > 0:
                total_score += word_score / len(synsets)
    return total_score

In [1301]:
# Opinión 1
print(classify_swn_gram(opinion1))

26.805654761904766


In [1302]:
# Opinión 2
print(classify_swn_gram(opinion2))

26.569603892348248


In [1303]:
# Opinión 3
print(classify_swn_gram(opinion3))

7.053113553113552


Notamos que la mejora es más bien leve.

---
- **Ejercicio 3:** Se pide extender lo que se ha hecho en los ejercicios anteriores, pero ampliado para todas las opiniones contenidas en un fichero .csv (textsSentimentsPNN.csv), donde cada opinión está anotada con su polaridad (positiva, negativa o neutra). Un ejemplo del formato del archivo se muestra a continuación, en la primera columna estaría el texto de la opinión y en la segunda el sentimiento (Positive, Negative o Neutral).  
 
    Utilizando de nuevo SentiWordNet se pide predecir la polaridad de cada mensaje. Una vez que se tenga la polaridad de cada mensaje, utilizar las métricas del paquete sklearn.metrics de la librería scikit-learn para obtener los valores de accuracy, precisión, recall y f-measure. En definitiva, se quiere poder cuantificar si se están clasificando bien las opiniones según su polaridad. Cuanto más cercano a 1 sea el valor que se obtiene con estas métricas, mejor estará clasificando cada mensaje.

In [1304]:
# Leemos el archivo ".csv" con pandas
texts_sentiments = pd.read_csv("data/textsSentimentsPNN.csv")

# Mostramos las 5 primeras filas
texts_sentiments.head(5)

,Text,Sentiment
0,Enjoying a beautiful day at the park!,Positive
1,Traffic was terrible this morning.,Negative
2,Just finished an amazing workout! 💪,Positive
3,Excited about the upcoming weekend getaway!,Positive
4,Trying out a new recipe for dinner tonight.,Neutral


In [1305]:
texts_sentiments["Sentiment"] == "Positive"

0      True
1     False
2      True
3      True
4     False
      ...  
62    False
63    False
64    False
65    False
66    False
Name: Sentiment, Length: 67, dtype: bool

In [1306]:
# Creamos un diccionario para codificar las etiquetas
dicc_encode = {
    "Neutral": 0,
    "Positive": 1,
    "Negative": 2
}

# Aplicamos la transformación y comprobamos el resultado
for key, value in dicc_encode.items():
    mask = (texts_sentiments["Sentiment"] == key)
    texts_sentiments.loc[mask, "Sentiment"] = value

texts_sentiments.head(5)

,Text,Sentiment
0,Enjoying a beautiful day at the park!,1
1,Traffic was terrible this morning.,2
2,Just finished an amazing workout! 💪,1
3,Excited about the upcoming weekend getaway!,1
4,Trying out a new recipe for dinner tonight.,0


In [ ]:
# Función para preprocesar cada texto
def preprocess_text(text):
    # Tokenizamos el texto en sentencias
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)

    # Tokenizamos cada sentencia en palabras,
    # eliminando a su vez la puntuación
    word_tokenized_np = [regexp_tokenize(s.lower(), r"[\w\d]+") for s in sent_tokenized]

    # Eliminamos las stopwords
    word_tokenized_np_nsw = []
    for s in word_tokenized_np:
        word_tokenized_np_nsw.append([w for w in s if w not in english_sw])
    
    # Lematizamos
    lemmatized_tokens = []
    for s in word_tokenized_np_nsw:
        lemmatized_tokens.append([WordNetLemmatizer().lemmatize(w) for w in s])

    return lemmatized_tokens

# Mostramos un ejemplo de cómo quedaría un texto preprocesado
print(f"Texto original: {texts_sentiments["Text"][0]}")
print(f"Texto preprocesado: {preprocess_text(texts_sentiments["Text"][0])}")

Texto original: Enjoying a beautiful day at the park!
Texto preprocesado: (['Enjoying a beautiful day at the park!'], [['Enjoying', 'beautiful', 'day', 'park']])


In [ ]:
# Función para obtener el score de un texto
def get_text_score(text):
    preprocessed = preprocess_text(text)
    tags = [nltk.pos_tag(s) for s in preprocessed]
    total_score = 0
    for s_idx in range(len(preprocessed)):
        sent_score = 0
        for w, t in tags[s_idx]:
            synset = simple_lesk(" ".join(preprocessed[s_idx]), w, tag2wn(t))
            if synset is not None:
                

# Mostramos un ejemplo de score para un texto
print(f"Texto: {texts_sentiments["Text"][0]}")
print(f"Score: {get_text_score(texts_sentiments["Text"][0])}")

Texto: Enjoying a beautiful day at the park!
Score: 0.023986486486486484


In [1309]:
# Obtenemos el vector de scores para todos los textos
y_pred = np.array([get_text_score(t) for t in texts_sentiments["Text"]], dtype=np.float32)
print(y_pred)

[0.02398649 0.         0.         0.         0.         0.
 0.00124008 0.00411184 0.         0.         0.         0.
 0.         0.00558036 0.00145688 0.         0.0094697  0.
 0.         0.         0.01779762 0.         0.01143293 0.
 0.         0.         0.         0.01651306 0.01358696 0.
 0.         0.00434783 0.         0.         0.         0.00526316
 0.00594406 0.         0.         0.         0.         0.
 0.01633523 0.01551227 0.         0.         0.         0.
 0.01015625 0.0078125  0.00372024 0.         0.00173611 0.003125
 0.00238715 0.         0.         0.00102041 0.         0.00107143
 0.         0.00138889 0.00214844 0.0010274  0.00437556 0.
 0.00810185]


In [1310]:
# Mostramos el rango de valores de las predicciones
y_pred_min = y_pred.min()
y_pred_max = y_pred.max()
print(f"Val. mínimo: {y_pred_min}; Val. máximo: {y_pred_max}")

Val. mínimo: 0.0; Val. máximo: 0.023986486718058586


In [1311]:
# Normalizamos las predicciones al rango [-1, 1]
y_pred_norm = 2 * ((y_pred - y_pred_min) / (y_pred_max - y_pred_min)) - 1
print(f"Val. mínimo: {y_pred_norm.min()}; Val. máximo: {y_pred_norm.max()}")
print("\nArray normalizado al intervalo [-1, 1]:")
print(y_pred_norm)

Val. mínimo: -1.0; Val. máximo: 1.0

Array normalizado al intervalo [-1, 1]:
[ 1.         -1.         -1.         -1.         -1.         -1.
 -0.89660186 -0.6571534  -1.         -1.         -1.         -1.
 -1.         -0.53470826 -0.87852526 -1.         -0.21041399 -1.
 -1.         -1.          0.48397052 -1.         -0.04671931 -1.
 -1.         -1.         -1.          0.37686336  0.13288426 -1.
 -1.         -0.63747704 -1.         -1.         -1.         -0.5611564
 -0.50438297 -1.         -1.         -1.         -1.         -1.
  0.36203575  0.29341698 -1.         -1.         -1.         -1.
 -0.15316904 -0.34859157 -0.6898055  -1.         -0.85524255 -0.7394366
 -0.8009585  -1.         -1.         -0.91491807 -1.         -0.91066396
 -1.         -0.8841941  -0.82086265 -0.9143353  -0.63516486 -1.
 -0.32446533]


In [1312]:
threshold = 0.7
y_pred_encoded = y_pred_norm.copy()
positive_pred = (y_pred_encoded > threshold)
negative_pred = (y_pred_encoded < threshold)
neutral_pred = (y_pred_encoded >= -threshold) & (y_pred_encoded <= threshold)
y_pred_encoded[positive_pred] = dicc_encode["Positive"]
y_pred_encoded[negative_pred] = dicc_encode["Negative"]
y_pred_encoded[neutral_pred] = dicc_encode["Neutral"]
y_pred_encoded = y_pred_encoded.astype(np.uint8)

print("Predicciones codificadas:")
print(y_pred_encoded)

Predicciones codificadas:
[1 2 2 2 2 2 2 0 2 2 2 2 2 0 2 2 0 2 2 2 0 2 0 2 2 2 2 0 0 2 2 0 2 2 2 0 0
 2 2 2 2 2 0 0 2 2 2 2 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 0]


In [ ]:
# Obtenemos las métricas
y_real = np.array(texts_sentiments["Sentiment"], dtype=np.uint8)
print(f"Accuracy: {accuracy_score(y_real, y_pred_encoded)}")
print(f"Precision: {precision_score(y_real, y_pred)}")
print(f"Recall: {recall_score(y_real, y_pred)}")
print(f"F-score: {f1_score(y_real, y_pred)}")

Accuracy: 0.1044776119402985
